# SDCR Interferometry Overview

This notebook provides a **read-only visualization** of Symmetry-Driven Coherence Restoration (SDCR)
applied to a minimal two-path interferometer model.

The purpose of this notebook is to **illustrate** results generated by the SDCR core
using fixed parameters and explicit recovery limits.

No parameter tuning or interactive controls are provided by design.

## Scope and Intent

- The interferometer is modeled as a two-level open quantum system.
- Dynamics follow standard Lindblad evolution.
- SDCR is applied as a **symmetry-selection operator** acting on reduced dynamics.
- A recovery configuration (symmetry disabled) is included explicitly.

This notebook:
- does **not** introduce new physics
- does **not** explore parameter space
- does **not** modify core logic

It mirrors the behavior shown in the SDCR reference implementation.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Ensure repo root is on path (safe for local clones)
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from core.recovery import build_selector, solve_with_recovery
from core.symmetry import pauli_z_symmetry
from core.observables import coherence_01, phase_01, time_series

from domains.interferometry.model import (
    InterferometerParams,
    build_interferometer_model,
    default_initial_state,
)

## Interferometer Model

The two interferometer paths are represented as basis states |0⟩ and |1⟩.

- Relative phase accumulation is encoded via a σᶻ Hamiltonian term.
- Dephasing is modeled using a standard Lindblad σᶻ operator.
- The initial state is an equal superposition of both paths.

All parameters below are **fixed** to provide a stable reference configuration.

In [ ]:
params = InterferometerParams(
    phase_rate=1.0,
    mixing_rate=0.0,
    dephasing_rate=0.3,
)

T_FINAL = 10.0
N_STEPS = 400

rho0 = default_initial_state()
H, L_ops = build_interferometer_model(params)

t_span = (0.0, T_FINAL)
t_eval = np.linspace(t_span[0], t_span[1], N_STEPS)

In [ ]:
projector = pauli_z_symmetry(dim=2)

# Baseline (symmetry disabled)
selector_baseline = build_selector(projector=projector, enabled=False)
t, rhos_base = solve_with_recovery(
    rho0=rho0,
    H=H,
    L_ops=L_ops,
    t_span=t_span,
    t_eval=t_eval,
    selector=selector_baseline,
)

# SDCR enabled
selector_sdcr = build_selector(projector=projector, enabled=True)
_, rhos_sdcr = solve_with_recovery(
    rho0=rho0,
    H=H,
    L_ops=L_ops,
    t_span=t_span,
    t_eval=t_eval,
    selector=selector_sdcr,
)

# Explicit recovery
selector_recovery = build_selector(projector=projector, enabled=False)
_, rhos_rec = solve_with_recovery(
    rho0=rho0,
    H=H,
    L_ops=L_ops,
    t_span=t_span,
    t_eval=t_eval,
    selector=selector_recovery,
)

## Extracted Observables

Two simple proxies are shown:

- **Visibility proxy**:  
  \\( 2|\rho_{01}| \\), bounded in [0,1] for the chosen initial state

- **Phase proxy**:  
  \\( \arg(\rho_{01}) \\)

These quantities are extracted directly from the reduced density matrix.

In [ ]:
vis_base = 2.0 * time_series(rhos_base, coherence_01)
vis_sdcr = 2.0 * time_series(rhos_sdcr, coherence_01)
vis_rec = 2.0 * time_series(rhos_rec, coherence_01)

ph_base = time_series(rhos_base, phase_01)
ph_sdcr = time_series(rhos_sdcr, phase_01)
ph_rec = time_series(rhos_rec, phase_01)

## Visualization

Baseline, SDCR-enabled, and recovery evolutions are shown together
to make recovery limits explicit.

In [ ]:
plt.rcParams.update(
    {
        "figure.figsize": (9, 4),
        "axes.grid": True,
        "grid.alpha": 0.3,
        "lines.linewidth": 2.0,
    }
)

In [ ]:
# Visibility
plt.figure()
plt.plot(t, vis_base, label="Baseline")
plt.plot(t, vis_sdcr, label="SDCR (symmetry on)")
plt.plot(t, vis_rec, "--", label="Recovery")
plt.xlabel("Time")
plt.ylabel("Visibility proxy  2|ρ₀₁|")
plt.title("SDCR Interferometry — Coherence / Visibility")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Phase
plt.figure()
plt.plot(t, ph_base, label="Baseline")
plt.plot(t, ph_sdcr, label="SDCR (symmetry on)")
plt.plot(t, ph_rec, "--", label="Recovery")
plt.xlabel("Time")
plt.ylabel("Phase proxy  arg(ρ₀₁)")
plt.title("SDCR Interferometry — Phase")
plt.legend()
plt.tight_layout()
plt.show()

## Notes

- When symmetry selection is disabled, SDCR reduces exactly to baseline evolution.
- Differences between baseline and SDCR-enabled curves vanish in the recovery limit.
- This notebook is intended as a **visual reference**, not an experimental interface.

For executable tests and null checks, see:
- `examples/null_test.py`
- `tests/test_interferometry_domain.py`